In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
import gc
import os

# --- CẤU HÌNH ---
# Hãy sửa đường dẫn cho đúng với máy của bạn
TRAIN_SEQ_PATH = '/kaggle/input/btl-ml/cafa-6-protein-function-prediction/Train/train_sequences.fasta' 
TRAIN_TERMS_PATH = '/kaggle/input/btl-ml/cafa-6-protein-function-prediction/Train/train_terms.tsv'
TEST_SEQ_PATH = '/kaggle/input/btl-ml/cafa-6-protein-function-prediction/Test/testsuperset.fasta'
OBO_PATH = '/kaggle/input/btl-ml/cafa-6-protein-function-prediction/Train/go-basic.obo'

BATCH_SIZE = 32      # Tăng lên 64 cho ổn định
EPOCHS = 20         # Tăng thời gian học
LEARNING_RATE = 0.001
NUM_LABELS = 1500    # Tăng số lượng nhãn để model học rộng hơn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang sử dụng thiết bị: {device}")

Đang sử dụng thiết bị: cuda


In [3]:
# --- 1. HÀM XỬ LÝ DỮ LIỆU ---

def load_fasta(path):
    """Đọc file FASTA và lấy ID chính xác"""
    sequences = {}
    current_id = None
    current_seq = []
    
    with open(path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if current_id:
                    sequences[current_id] = ''.join(current_seq)
                
                header = line[1:]
                # Xử lý header dạng 'sp|ID|Name' hoặc '>ID'
                if '|' in header:
                    parts = header.split('|')
                    if len(parts) > 1:
                        current_id = parts[1]
                    else:
                        current_id = header.split()[0]
                else:
                    current_id = header.split()[0]
                    
                current_seq = []
            else:
                current_seq.append(line)
        if current_id:
            sequences[current_id] = ''.join(current_seq)
    return sequences

def get_dipeptide_composition(sequence):
    """
    Tạo vector đặc trưng 400 chiều từ tần suất cặp axit amin.
    Input: Chuỗi protein. Output: Vector (400,)
    """
    aa_list = 'ACDEFGHIKLMNPQRSTVWY'
    aa_map = {aa: i for i, aa in enumerate(aa_list)}
    
    dipeptide_counts = np.zeros((20, 20), dtype=np.float32)
    length = len(sequence)
    
    if length < 2: 
        return dipeptide_counts.flatten()
    
    for i in range(length - 1):
        a1 = sequence[i]
        a2 = sequence[i+1]
        if a1 in aa_map and a2 in aa_map:
            dipeptide_counts[aa_map[a1], aa_map[a2]] += 1
            
    return dipeptide_counts.flatten() / (length - 1)


In [4]:
# --- 2. CHUẨN BỊ DATASET ---

print("1. Đang đọc dữ liệu Train...")
train_seqs = load_fasta(TRAIN_SEQ_PATH)
train_terms = pd.read_csv(TRAIN_TERMS_PATH, sep='\t')

# Lọc top N nhãn phổ biến nhất
top_terms = train_terms['term'].value_counts().head(NUM_LABELS).index.tolist()
term_to_idx = {term: i for i, term in enumerate(top_terms)}
idx_to_term = {i: term for term, i in term_to_idx.items()} # Dùng để map ngược lại khi dự đoán

# Lọc dữ liệu train chỉ giữ lại các dòng thuộc top terms
train_terms_filtered = train_terms[train_terms['term'].isin(top_terms)]
protein_to_labels = train_terms_filtered.groupby('EntryID')['term'].apply(list).to_dict()

# Lấy danh sách protein hợp lệ (có cả sequence và label)
valid_ids = [pid for pid in train_seqs.keys() if pid in protein_to_labels]
print(f"Số lượng protein hợp lệ: {len(valid_ids)}")

class CAFA6Dataset(Dataset):
    def __init__(self, protein_ids, seq_dict, label_dict, term_map, num_classes):
        self.protein_ids = protein_ids
        self.seq_dict = seq_dict
        self.label_dict = label_dict
        self.term_map = term_map
        self.num_classes = num_classes

    def __len__(self):
        return len(self.protein_ids)

    def __getitem__(self, idx):
        pid = self.protein_ids[idx]
        seq = self.seq_dict[pid]
        
        # Tạo feature 400 chiều
        features = get_dipeptide_composition(seq)
        
        # Tạo label one-hot
        labels = np.zeros(self.num_classes, dtype=np.float32)
        if pid in self.label_dict:
            for term in self.label_dict[pid]:
                if term in self.term_map:
                    labels[self.term_map[term]] = 1.0
        
        return torch.tensor(features, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)

# Chia tập train/val
train_ids, val_ids = train_test_split(valid_ids, test_size=0.1, random_state=42)

train_dataset = CAFA6Dataset(train_ids, train_seqs, protein_to_labels, term_to_idx, NUM_LABELS)
val_dataset = CAFA6Dataset(val_ids, train_seqs, protein_to_labels, term_to_idx, NUM_LABELS)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)



1. Đang đọc dữ liệu Train...
Số lượng protein hợp lệ: 76297


In [5]:
# --- 3. MÔ HÌNH RES-MLP ---

class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout_rate=0.4):
        super(ResidualBlock, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        
    def forward(self, x):
        return x + self.layer(x) # Skip connection

class ResMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ResMLP, self).__init__()
        self.entry = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.blocks = nn.Sequential(
            ResidualBlock(1024),
            ResidualBlock(1024)
        )
        self.head = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.entry(x)
        x = self.blocks(x)
        return self.head(x)



In [6]:
# --- 4. TRAINING LOOP ---

print("2. Bắt đầu huấn luyện...")
# Input dim = 400 (Di-peptide)
model = ResMLP(input_dim=400, num_classes=NUM_LABELS).to(device)

criterion = nn.BCEWithLogitsLoss()
# Dùng AdamW + Weight Decay để chống Overfitting
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
# Tự động giảm tốc độ học nếu không tiến bộ
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

best_val_loss = float('inf')

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for features, labels in val_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    
    scheduler.step(avg_val_loss)
    
    # Lưu model tốt nhất
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_model.pth')
            
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

print("Huấn luyện hoàn tất. Đã lưu 'best_model.pth'.")

2. Bắt đầu huấn luyện...


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20 | Train Loss: 0.0168 | Val Loss: 0.0149
Epoch 2/20 | Train Loss: 0.0148 | Val Loss: 0.0144
Epoch 3/20 | Train Loss: 0.0141 | Val Loss: 0.0141
Epoch 4/20 | Train Loss: 0.0135 | Val Loss: 0.0139
Epoch 5/20 | Train Loss: 0.0129 | Val Loss: 0.0139
Epoch 6/20 | Train Loss: 0.0125 | Val Loss: 0.0139
Epoch 7/20 | Train Loss: 0.0121 | Val Loss: 0.0140
Epoch 8/20 | Train Loss: 0.0118 | Val Loss: 0.0140
Epoch 9/20 | Train Loss: 0.0115 | Val Loss: 0.0142
Epoch 10/20 | Train Loss: 0.0107 | Val Loss: 0.0140
Epoch 11/20 | Train Loss: 0.0104 | Val Loss: 0.0141
Epoch 12/20 | Train Loss: 0.0102 | Val Loss: 0.0142
Epoch 13/20 | Train Loss: 0.0101 | Val Loss: 0.0141
Epoch 14/20 | Train Loss: 0.0097 | Val Loss: 0.0144
Epoch 15/20 | Train Loss: 0.0096 | Val Loss: 0.0143
Epoch 16/20 | Train Loss: 0.0095 | Val Loss: 0.0143
Epoch 17/20 | Train Loss: 0.0094 | Val Loss: 0.0145
Epoch 18/20 | Train Loss: 0.0092 | Val Loss: 0.0145
Epoch 19/20 | Train Loss: 0.0091 | Val Loss: 0.0143
Epoch 20/20 | Train L

In [7]:
# --- 5. TẠO FILE SUBMISSION ---
print("3. Đang đọc file Test và dự đoán...")

# Load lại model tốt nhất
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

test_seqs = load_fasta(TEST_SEQ_PATH)
TEMP_SUBMISSION_FILE = 'submission_temp.tsv'

with open(TEMP_SUBMISSION_FILE, 'w') as f:
    count = 0
    # Xử lý từng protein trong tập test
    for pid, seq in test_seqs.items():
        if len(seq) < 2: continue # Bỏ qua chuỗi quá ngắn
        
        features = get_dipeptide_composition(seq)
        features_tensor = torch.tensor([features], dtype=torch.float32).to(device)
        
        with torch.no_grad():
            logits = model(features_tensor)
            probs = torch.sigmoid(logits).cpu().numpy()[0]
        
        # Lấy các term có điểm > 0.005 (Ngưỡng thấp để giữ lại nhiều ứng viên cho bước sau)
        # Chỉ lấy top 50 dự đoán cao nhất cho mỗi protein để giảm dung lượng
        top_indices = np.argsort(probs)[::-1][:50]
        
        for idx in top_indices:
            score = probs[idx]
            if score > 0.005: 
                term_id = idx_to_term[idx] # Chuyển từ index số về GO ID (GO:000...)
                f.write(f"{pid}\t{term_id}\t{score:.3f}\n")
        
        count += 1
        if count % 2000 == 0:
            print(f"Đã dự đoán {count} protein...")

print(f"Dự đoán xong. File tạm: {TEMP_SUBMISSION_FILE}")

3. Đang đọc file Test và dự đoán...


/tmp/ipykernel_47/659956551.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  features_tensor = torch.tensor([features], dtype=torch.float32).to(device)


Đã dự đoán 2000 protein...
Đã dự đoán 4000 protein...
Đã dự đoán 6000 protein...
Đã dự đoán 8000 protein...
Đã dự đoán 10000 protein...
Đã dự đoán 12000 protein...
Đã dự đoán 14000 protein...
Đã dự đoán 16000 protein...
Đã dự đoán 18000 protein...
Đã dự đoán 20000 protein...
Đã dự đoán 22000 protein...
Đã dự đoán 24000 protein...
Đã dự đoán 26000 protein...
Đã dự đoán 28000 protein...
Đã dự đoán 30000 protein...
Đã dự đoán 32000 protein...
Đã dự đoán 34000 protein...
Đã dự đoán 36000 protein...
Đã dự đoán 38000 protein...
Đã dự đoán 40000 protein...
Đã dự đoán 42000 protein...
Đã dự đoán 44000 protein...
Đã dự đoán 46000 protein...
Đã dự đoán 48000 protein...
Đã dự đoán 50000 protein...
Đã dự đoán 52000 protein...
Đã dự đoán 54000 protein...
Đã dự đoán 56000 protein...
Đã dự đoán 58000 protein...
Đã dự đoán 60000 protein...
Đã dự đoán 62000 protein...
Đã dự đoán 64000 protein...
Đã dự đoán 66000 protein...
Đã dự đoán 68000 protein...
Đã dự đoán 70000 protein...
Đã dự đoán 72000 protein

In [8]:
!pip install goatools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 94.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 13.8 MB/s eta 0:00:00


In [9]:
# --- 6. HẬU XỬ LÝ VỚI OBO FILE ---
from goatools.obo_parser import GODag
from tqdm import tqdm
import os

print("4. Bắt đầu Post-processing (Lan truyền điểm)...")
FINAL_OUTPUT = 'submission.tsv'

# Kiểm tra file OBO
if not os.path.exists(OBO_PATH):
    print("CẢNH BÁO: Không tìm thấy file OBO. Sẽ dùng file tạm làm kết quả cuối cùng.")
    os.rename(TEMP_SUBMISSION_FILE, FINAL_OUTPUT)
else:
    print("Đang load cây phả hệ GO...")
    godag = GODag(OBO_PATH)

    def propagate_scores(df_group, godag):
        # Lấy danh sách term và score hiện tại
        current_scores = dict(zip(df_group['GO_Term'], df_group['Score']))
        new_scores = current_scores.copy()
        
        for go_id, score in current_scores.items():
            if go_id not in godag: continue
            
            term_obj = godag[go_id]
            ancestors = term_obj.get_all_parents()
            
            # Cha phải có điểm ít nhất bằng điểm của Con
            for ancestor in ancestors:
                anc_score_old = new_scores.get(ancestor, 0.0)
                new_scores[ancestor] = max(anc_score_old, score)
                
        return [[pid, term, score] for term, score in new_scores.items() if score >= 0.01]

    # Đọc file tạm
    sub_df = pd.read_csv(TEMP_SUBMISSION_FILE, sep='\t', names=['ProteinID', 'GO_Term', 'Score'])
    
    final_data = []
    
    print("Đang xử lý logic Cha-Con cho từng protein...")
    for pid, group in tqdm(sub_df.groupby('ProteinID')):
        refined_rows = propagate_scores(group, godag)
        for row in refined_rows:
            # row = [pid, term, score]
            final_data.append(row)
            
    # Lưu file cuối cùng
    print("Đang lưu file kết quả cuối cùng...")
    result_df = pd.DataFrame(final_data, columns=['ProteinID', 'GO_Term', 'Score'])
    
    # Format điểm số 3 số lẻ
    result_df['Score'] = result_df['Score'].map(lambda x: '{:.3f}'.format(x))
    
    # Lưu tsv không header
    result_df.to_csv(FINAL_OUTPUT, sep='\t', index=False, header=False)
    
    print(f"XONG! File nộp bài của bạn là: {FINAL_OUTPUT}")

4. Bắt đầu Post-processing (Lan truyền điểm)...
Đang load cây phả hệ GO...
/kaggle/input/btl-ml/cafa-6-protein-function-prediction/Train/go-basic.obo: fmt(1.2) rel(2025-06-01) 43,448 Terms
Đang xử lý logic Cha-Con cho từng protein...


100%|██████████| 224309/224309 [02:18<00:00, 1618.67it/s]


Đang lưu file kết quả cuối cùng...
XONG! File nộp bài của bạn là: submission.tsv
